# Module 03: Advanced Experimental Designs

**Estimated Time**: 45 minutes

## Learning Objectives

By the end of this module, you will be able to:

1. **Distinguish** between true experimental, quasi-experimental, and observational designs
2. **Select** appropriate experimental designs for different research contexts
3. **Implement** within-subjects and between-subjects designs with proper analysis
4. **Apply** crossover designs with appropriate washout periods
5. **Analyze** interrupted time series data to detect intervention effects
6. **Understand** regression discontinuity designs for causal inference
7. **Identify** and leverage natural experiments in observational data
8. **Conduct** sensitivity analyses to test robustness of findings

## Why This Matters

While **Randomized Controlled Trials (RCTs)** are the gold standard for causal inference, they're often:
- **Impractical** (can't randomize people to smoke cigarettes)
- **Unethical** (can't withhold beneficial treatments)
- **Expensive** (require large samples and long follow-ups)
- **Limited** (may not generalize to real-world settings)

Advanced experimental designs allow researchers to:
- Make **causal claims** in observational settings
- **Reduce bias** through clever design choices
- **Increase statistical power** with repeated measures
- **Answer important questions** that RCTs cannot address

This module equips you with a sophisticated toolkit for designing rigorous studies when perfect experiments aren't possible.

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_rel, ttest_ind, f_oneway
import warnings

warnings.filterwarnings("ignore")

# Set style
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_palette("husl")

# Set random seed for reproducibility
np.random.seed(42)

# Create output directory
import os

os.makedirs("outputs/module_03", exist_ok=True)

print("✓ Libraries imported successfully")
print("✓ Output directory created")

## 1. Experimental Design Landscape

### The Hierarchy of Evidence

Research designs vary in their ability to support causal claims:

```
STRONGEST EVIDENCE ↓

1. Systematic Reviews & Meta-Analyses
   └─ Synthesize multiple RCTs

2. Randomized Controlled Trials (RCTs)
   └─ Random assignment eliminates confounding

3. Quasi-Experimental Designs
   ├─ Regression Discontinuity
   ├─ Interrupted Time Series
   ├─ Difference-in-Differences
   └─ Natural Experiments

4. Cohort Studies
   └─ Follow groups over time

5. Case-Control Studies
   └─ Compare cases to controls retrospectively

6. Cross-Sectional Studies
   └─ Snapshot at one time point

7. Case Reports & Expert Opinion
   └─ Individual observations

WEAKEST EVIDENCE ↑
```

### Key Dimensions of Experimental Design

| Dimension | Options | Trade-offs |
|-----------|---------|------------|
| **Assignment** | Random vs. Non-random | Eliminates confounding vs. Practical |
| **Comparison** | Between-subjects vs. Within-subjects | Independent vs. Powerful |
| **Timing** | Cross-sectional vs. Longitudinal | Quick vs. Causal |
| **Control** | Active vs. Placebo vs. None | Specific vs. General |
| **Blinding** | Single vs. Double vs. Triple | Reduces bias vs. Practical |

Let's explore advanced designs that maximize causal inference when RCTs aren't feasible.

In [ ]:
# Visualize the design decision tree
design_data = {
    "Design Type": [
        "RCT",
        "RCT",
        "Quasi-Exp",
        "Quasi-Exp",
        "Quasi-Exp",
        "Observational",
        "Observational",
    ],
    "Specific Design": [
        "Between-Subjects",
        "Within-Subjects",
        "Regression Discontinuity",
        "Interrupted Time Series",
        "Natural Experiment",
        "Cohort",
        "Cross-Sectional",
    ],
    "Causal Strength": [95, 90, 80, 75, 70, 50, 30],
    "Feasibility": [30, 40, 70, 75, 80, 85, 95],
}

df_designs = pd.DataFrame(design_data)

fig, ax = plt.subplots(figsize=(12, 6))

colors = {"RCT": "#2E86AB", "Quasi-Exp": "#A23B72", "Observational": "#F18F01"}
for design_type in df_designs["Design Type"].unique():
    subset = df_designs[df_designs["Design Type"] == design_type]
    ax.scatter(
        subset["Feasibility"],
        subset["Causal Strength"],
        s=200,
        alpha=0.6,
        label=design_type,
        color=colors[design_type],
    )

    for idx, row in subset.iterrows():
        ax.annotate(
            row["Specific Design"],
            (row["Feasibility"], row["Causal Strength"]),
            xytext=(5, 5),
            textcoords="offset points",
            fontsize=8,
        )

ax.set_xlabel("Feasibility (Ease of Implementation)", fontsize=12, fontweight="bold")
ax.set_ylabel("Causal Inference Strength", fontsize=12, fontweight="bold")
ax.set_title("The Trade-off: Causal Strength vs. Feasibility", fontsize=14, fontweight="bold")
ax.legend(title="Design Category", loc="lower left")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs/module_03/design_tradeoff.png", dpi=300, bbox_inches="tight")
plt.show()

print("📊 The ideal design balances causal strength with practical feasibility")

## 2. Between-Subjects vs. Within-Subjects Designs

### Between-Subjects (Independent Groups)

**Design**: Different participants in each condition

**Advantages**:
- No carryover effects
- No practice effects
- No demand characteristics from repeated exposure

**Disadvantages**:
- Requires larger sample size
- Individual differences add noise
- Lower statistical power

### Within-Subjects (Repeated Measures)

**Design**: Same participants in all conditions

**Advantages**:
- Controls for individual differences
- Requires fewer participants
- Higher statistical power

**Disadvantages**:
- Carryover effects possible
- Practice/fatigue effects
- Attrition concerns

### Power Comparison

Let's demonstrate why within-subjects designs are more powerful.

In [ ]:
# Simulate the power advantage of within-subjects designs


def simulate_experiment(n_participants, effect_size, design="between", n_simulations=1000):
    """
    Simulate experiments to compare power of different designs.

    Parameters:
    - n_participants: Number of participants
    - effect_size: Cohen's d
    - design: 'between' or 'within'
    - n_simulations: Number of simulations to run

    Returns:
    - proportion of significant results (empirical power)
    """
    significant_count = 0

    for _ in range(n_simulations):
        if design == "between":
            # Between-subjects: independent groups
            group1 = np.random.normal(0, 1, n_participants)
            group2 = np.random.normal(effect_size, 1, n_participants)
            _, p_value = ttest_ind(group1, group2)

        else:  # within-subjects
            # Within-subjects: same participants, correlated measures
            # Individual differences (baseline ability)
            baseline = np.random.normal(0, 1, n_participants)

            # Condition 1: baseline + random noise
            condition1 = baseline + np.random.normal(0, 0.5, n_participants)

            # Condition 2: baseline + effect + random noise
            condition2 = baseline + effect_size + np.random.normal(0, 0.5, n_participants)

            _, p_value = ttest_rel(condition1, condition2)

        if p_value < 0.05:
            significant_count += 1

    return significant_count / n_simulations


# Compare power across sample sizes
sample_sizes = range(10, 101, 10)
effect_size = 0.5  # Medium effect

power_between = [simulate_experiment(n, effect_size, "between", 500) for n in sample_sizes]
power_within = [simulate_experiment(n, effect_size, "within", 500) for n in sample_sizes]

# Visualize power comparison
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(
    sample_sizes,
    power_between,
    "o-",
    linewidth=2,
    markersize=8,
    label="Between-Subjects",
    color="#E63946",
)
ax.plot(
    sample_sizes,
    power_within,
    "s-",
    linewidth=2,
    markersize=8,
    label="Within-Subjects",
    color="#06A77D",
)
ax.axhline(y=0.80, color="gray", linestyle="--", linewidth=1.5, label="Target Power (80%)")

ax.set_xlabel("Sample Size (N per group)", fontsize=12, fontweight="bold")
ax.set_ylabel("Statistical Power", fontsize=12, fontweight="bold")
ax.set_title(
    f"Power Comparison: Within vs. Between (Effect Size d = {effect_size})",
    fontsize=14,
    fontweight="bold",
)
ax.legend(loc="lower right", fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig("outputs/module_03/power_comparison.png", dpi=300, bbox_inches="tight")
plt.show()

# Find required sample sizes for 80% power
n_between_80 = next((n for n, p in zip(sample_sizes, power_between) if p >= 0.80), None)
n_within_80 = next((n for n, p in zip(sample_sizes, power_within) if p >= 0.80), None)

print(f"\n📊 Power Analysis Results (d = {effect_size}):")
print(f"\nBetween-subjects requires N ≈ {n_between_80} per group for 80% power")
print(f"Within-subjects requires N ≈ {n_within_80} participants for 80% power")
print(f"\n💡 Within-subjects design is ~{n_between_80/n_within_80:.1f}x more efficient!")

### When to Use Each Design

**Use Between-Subjects when**:
- Exposure to one condition affects responses to others (e.g., learning)
- The intervention permanently changes participants (e.g., training)
- You're studying stable traits (e.g., personality)
- Repeated testing is impractical or expensive

**Use Within-Subjects when**:
- Individual differences are large (increases power)
- Sample size is limited (clinical populations)
- Conditions are clearly distinguishable (no confusion)
- Order effects can be controlled (counterbalancing)

## 3. Crossover Designs

**Crossover designs** are a special type of within-subjects design where participants receive treatments in different sequences.

### Classic 2×2 Crossover Design

```
Group 1: A → [washout] → B
Group 2: B → [washout] → A
```

**Key Feature**: The **washout period** allows the first treatment's effects to dissipate before the second treatment.

### Advantages
1. Controls for individual differences (within-subjects)
2. Controls for order effects (counterbalancing)
3. High statistical power
4. Each participant serves as their own control

### Challenges
1. **Carryover effects**: Treatment A affects response to Treatment B
2. **Washout determination**: How long is sufficient?
3. **Attrition**: Longer studies = more dropout
4. **Period effects**: Changes over time confound treatment effects

### Example: Medication Trial

In [ ]:
# Simulate a crossover trial for two pain medications

np.random.seed(123)
n_participants = 40

# Individual baseline pain levels (person-specific)
baseline_pain = np.random.normal(50, 10, n_participants)

# True treatment effects
effect_drug_A = -15  # Reduces pain by 15 points
effect_drug_B = -10  # Reduces pain by 10 points

# Assign to sequences
sequence = np.random.choice(["A-B", "B-A"], size=n_participants)

# Simulate outcomes
data_crossover = []

for i in range(n_participants):
    person_baseline = baseline_pain[i]

    if sequence[i] == "A-B":
        # Period 1: Drug A
        period1_pain = person_baseline + effect_drug_A + np.random.normal(0, 5)
        # Period 2: Drug B (after washout)
        period2_pain = person_baseline + effect_drug_B + np.random.normal(0, 5)

        data_crossover.append(
            {
                "Participant": i + 1,
                "Sequence": "A→B",
                "Period_1_Treatment": "Drug A",
                "Period_1_Pain": period1_pain,
                "Period_2_Treatment": "Drug B",
                "Period_2_Pain": period2_pain,
            }
        )
    else:
        # Period 1: Drug B
        period1_pain = person_baseline + effect_drug_B + np.random.normal(0, 5)
        # Period 2: Drug A (after washout)
        period2_pain = person_baseline + effect_drug_A + np.random.normal(0, 5)

        data_crossover.append(
            {
                "Participant": i + 1,
                "Sequence": "B→A",
                "Period_1_Treatment": "Drug B",
                "Period_1_Pain": period1_pain,
                "Period_2_Treatment": "Drug A",
                "Period_2_Pain": period2_pain,
            }
        )

df_crossover = pd.DataFrame(data_crossover)

print("Crossover Trial Data (first 10 participants):")
print(df_crossover.head(10))

# Reshape for analysis
df_long = pd.concat(
    [
        df_crossover[["Participant", "Sequence", "Period_1_Treatment", "Period_1_Pain"]]
        .rename(columns={"Period_1_Treatment": "Treatment", "Period_1_Pain": "Pain_Score"})
        .assign(Period=1),
        df_crossover[["Participant", "Sequence", "Period_2_Treatment", "Period_2_Pain"]]
        .rename(columns={"Period_2_Treatment": "Treatment", "Period_2_Pain": "Pain_Score"})
        .assign(Period=2),
    ],
    ignore_index=True,
)

# Analyze
pain_A = df_long[df_long["Treatment"] == "Drug A"]["Pain_Score"]
pain_B = df_long[df_long["Treatment"] == "Drug B"]["Pain_Score"]

t_stat, p_value = ttest_ind(pain_A, pain_B)

print(f"\n📊 Crossover Trial Results:")
print(f"Mean pain with Drug A: {pain_A.mean():.1f} (SD = {pain_A.std():.1f})")
print(f"Mean pain with Drug B: {pain_B.mean():.1f} (SD = {pain_B.std():.1f})")
print(f"Difference: {pain_B.mean() - pain_A.mean():.1f} points")
print(f"t-test: t = {t_stat:.3f}, p = {p_value:.4f}")

if p_value < 0.05:
    print(f"\n✓ Drug A is significantly more effective than Drug B (p < .05)")
else:
    print(f"\n✗ No significant difference between drugs (p ≥ .05)")

In [ ]:
# Visualize crossover results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Spaghetti plot (individual trajectories)
for seq in df_crossover["Sequence"].unique():
    subset = df_crossover[df_crossover["Sequence"] == seq]
    color = "#E63946" if seq == "A→B" else "#06A77D"

    for _, row in subset.iterrows():
        axes[0].plot(
            [1, 2],
            [row["Period_1_Pain"], row["Period_2_Pain"]],
            color=color,
            alpha=0.3,
            linewidth=1,
        )

# Add sequence means
for seq, color, label in [("A→B", "#E63946", "Sequence: A→B"), ("B→A", "#06A77D", "Sequence: B→A")]:
    subset = df_crossover[df_crossover["Sequence"] == seq]
    means = [subset["Period_1_Pain"].mean(), subset["Period_2_Pain"].mean()]
    axes[0].plot(
        [1, 2], means, "o-", color=color, linewidth=3, markersize=10, label=label, alpha=0.8
    )

axes[0].set_xlabel("Period", fontsize=12, fontweight="bold")
axes[0].set_ylabel("Pain Score", fontsize=12, fontweight="bold")
axes[0].set_title("Individual Trajectories by Sequence", fontsize=13, fontweight="bold")
axes[0].set_xticks([1, 2])
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Panel 2: Treatment comparison
treatment_summary = df_long.groupby("Treatment")["Pain_Score"].agg(["mean", "std", "count"])
treatment_summary["se"] = treatment_summary["std"] / np.sqrt(treatment_summary["count"])

x_pos = [0, 1]
bars = axes[1].bar(
    x_pos,
    treatment_summary["mean"],
    yerr=treatment_summary["se"] * 1.96,  # 95% CI
    capsize=10,
    color=["#E63946", "#06A77D"],
    alpha=0.7,
    edgecolor="black",
    linewidth=1.5,
)

axes[1].set_ylabel("Pain Score (Lower = Better)", fontsize=12, fontweight="bold")
axes[1].set_title("Treatment Comparison\n(Mean ± 95% CI)", fontsize=13, fontweight="bold")
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(["Drug A", "Drug B"])
axes[1].grid(True, alpha=0.3, axis="y")

# Add significance indicator
if p_value < 0.05:
    y_max = max(treatment_summary["mean"]) + 5
    axes[1].plot([0, 1], [y_max, y_max], "k-", linewidth=1.5)
    axes[1].text(0.5, y_max + 1, f"p = {p_value:.3f}", ha="center", fontsize=10, fontweight="bold")

plt.tight_layout()
plt.savefig("outputs/module_03/crossover_results.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n💡 The crossover design allowed each participant to try both drugs,")
print("   controlling for individual differences in pain sensitivity.")

## 4. Interrupted Time Series (ITS) Analysis

**Interrupted Time Series** designs evaluate interventions by comparing trends before and after an event.

### Structure
```
Observations: ─────────────────|INTERVENTION|─────────────────
              Before (baseline)              After (treatment)
```

### What ITS Detects
1. **Level change**: Immediate jump after intervention
2. **Slope change**: Change in trend after intervention

### Statistical Model

$$Y_t = \beta_0 + \beta_1 \cdot \text{Time}_t + \beta_2 \cdot \text{Intervention}_t + \beta_3 \cdot \text{Time After}_t + \varepsilon_t$$

Where:
- $\beta_1$ = baseline trend
- $\beta_2$ = level change (immediate effect)
- $\beta_3$ = slope change (sustained effect)

### Example: Policy Intervention on Hospital Infections

In [ ]:
# Simulate hospital infection rates before and after hand hygiene policy

np.random.seed(456)

# Time points
n_before = 24  # 24 months before
n_after = 24  # 24 months after
n_total = n_before + n_after

time = np.arange(1, n_total + 1)
intervention = (time > n_before).astype(int)
time_after = np.maximum(0, time - n_before)

# True parameters
beta_0 = 15.0  # Baseline intercept (15 infections per 1000 patient-days)
beta_1 = 0.2  # Baseline trend (slowly increasing)
beta_2 = -5.0  # Immediate drop after intervention
beta_3 = -0.3  # Improved trend after intervention (decreasing)

# Generate infection rates
infection_rate = (
    beta_0
    + beta_1 * time
    + beta_2 * intervention
    + beta_3 * time_after
    + np.random.normal(0, 1.5, n_total)
)

# Create dataframe
df_its = pd.DataFrame(
    {
        "Month": time,
        "Intervention": intervention,
        "Time_After": time_after,
        "Infection_Rate": infection_rate,
        "Phase": ["Before" if i == 0 else "After" for i in intervention],
    }
)

print("Interrupted Time Series Data:")
print(df_its.head(10))
print("\n...\n")
print(df_its.tail(10))

In [ ]:
# Fit ITS regression model
from scipy.stats import linregress

# Using statsmodels for proper regression
try:
    import statsmodels.api as sm

    # Prepare design matrix
    X = df_its[["Month", "Intervention", "Time_After"]]
    X = sm.add_constant(X)
    y = df_its["Infection_Rate"]

    # Fit model
    model = sm.OLS(y, X).fit()

    print("\n" + "=" * 70)
    print("INTERRUPTED TIME SERIES REGRESSION RESULTS")
    print("=" * 70)
    print(model.summary())

    # Extract coefficients
    coefs = model.params
    pvals = model.pvalues

    print("\n" + "=" * 70)
    print("INTERPRETATION")
    print("=" * 70)

    print(f"\n1. Baseline Trend (β₁ = {coefs['Month']:.3f}, p = {pvals['Month']:.4f}):")
    if pvals["Month"] < 0.05:
        direction = "increasing" if coefs["Month"] > 0 else "decreasing"
        print(f"   Infection rates were significantly {direction} before intervention")
    else:
        print(f"   No significant trend before intervention")

    print(f"\n2. Level Change (β₂ = {coefs['Intervention']:.3f}, p = {pvals['Intervention']:.4f}):")
    if pvals["Intervention"] < 0.05:
        direction = "drop" if coefs["Intervention"] < 0 else "jump"
        print(
            f"   Immediate {direction} of {abs(coefs['Intervention']):.2f} infections after policy"
        )
    else:
        print(f"   No immediate change after intervention")

    print(f"\n3. Slope Change (β₃ = {coefs['Time_After']:.3f}, p = {pvals['Time_After']:.4f}):")
    if pvals["Time_After"] < 0.05:
        direction = "improved" if coefs["Time_After"] < 0 else "worsened"
        print(
            f"   Trend {direction} by {abs(coefs['Time_After']):.3f} infections per month after policy"
        )
    else:
        print(f"   No change in trend after intervention")

    # Generate predictions
    df_its["Predicted"] = model.predict(X)

    # Counterfactual: what would have happened without intervention?
    X_counterfactual = df_its[["Month", "Intervention", "Time_After"]].copy()
    X_counterfactual["Intervention"] = 0
    X_counterfactual["Time_After"] = 0
    X_counterfactual = sm.add_constant(X_counterfactual)
    df_its["Counterfactual"] = model.predict(X_counterfactual)

    statsmodels_available = True

except ImportError:
    print("\n⚠ statsmodels not available. Using simplified linear regression.")
    statsmodels_available = False

    # Simplified approach using scipy
    X_simple = np.column_stack([df_its["Month"], df_its["Intervention"], df_its["Time_After"]])
    # Note: This is a simplified version and won't give proper statistics

In [ ]:
# Visualize ITS results
fig, ax = plt.subplots(figsize=(14, 7))

# Plot observed data
before_data = df_its[df_its["Phase"] == "Before"]
after_data = df_its[df_its["Phase"] == "After"]

ax.scatter(
    before_data["Month"],
    before_data["Infection_Rate"],
    color="#E63946",
    s=80,
    alpha=0.6,
    label="Before Intervention",
    zorder=3,
)
ax.scatter(
    after_data["Month"],
    after_data["Infection_Rate"],
    color="#06A77D",
    s=80,
    alpha=0.6,
    label="After Intervention",
    zorder=3,
)

if statsmodels_available:
    # Plot fitted line
    ax.plot(
        df_its["Month"],
        df_its["Predicted"],
        color="#1D3557",
        linewidth=3,
        label="Fitted Model",
        zorder=4,
    )

    # Plot counterfactual (what would have happened without intervention)
    ax.plot(
        after_data["Month"],
        after_data["Counterfactual"],
        color="gray",
        linewidth=2,
        linestyle="--",
        label="Counterfactual (no intervention)",
        zorder=2,
    )

# Mark intervention point
ax.axvline(
    x=n_before, color="black", linestyle="-", linewidth=2, label="Intervention Start", zorder=1
)

# Shade regions
ax.axvspan(0, n_before, alpha=0.1, color="red", zorder=0)
ax.axvspan(n_before, n_total, alpha=0.1, color="green", zorder=0)

ax.set_xlabel("Month", fontsize=13, fontweight="bold")
ax.set_ylabel("Hospital Infection Rate\n(per 1000 patient-days)", fontsize=13, fontweight="bold")
ax.set_title(
    "Interrupted Time Series: Effect of Hand Hygiene Policy on Infection Rates",
    fontsize=14,
    fontweight="bold",
)
ax.legend(loc="upper right", fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs/module_03/its_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

if statsmodels_available:
    # Calculate effect size
    effect_at_end = (
        df_its[df_its["Month"] == n_total]["Counterfactual"].values[0]
        - df_its[df_its["Month"] == n_total]["Predicted"].values[0]
    )

    print(f"\n📊 Policy Impact Summary:")
    print(f"\nImmediate effect: {-coefs['Intervention']:.2f} fewer infections")
    print(f"Total effect by month {n_total}: {effect_at_end:.2f} fewer infections")
    print(f"\n💡 The ITS design provides strong evidence of policy effectiveness")
    print(f"   by comparing observed trends to counterfactual projections.")

### Strengths and Limitations of ITS

**Strengths**:
✓ No control group needed  
✓ Can evaluate population-level interventions  
✓ Controls for pre-existing trends  
✓ Transparent visual analysis  

**Limitations**:
✗ Assumes no other changes occurred at intervention time  
✗ Requires sufficient pre- and post-intervention observations (≥8 each)  
✗ Vulnerable to autocorrelation (consecutive observations correlated)  
✗ Cannot rule out confounding events  

## 5. Regression Discontinuity Design (RDD)

**Regression Discontinuity** exploits assignment rules based on a threshold to identify causal effects.

### Key Idea
When treatment is assigned based on a cutoff:
- People just above the threshold receive treatment
- People just below the threshold do not
- These groups are nearly identical except for treatment

### Example: Scholarship on Academic Performance

Students with entrance exam scores ≥70 receive a scholarship. Does the scholarship improve GPA?

In [ ]:
# Simulate regression discontinuity design

np.random.seed(789)
n_students = 500

# Entrance exam scores (running variable)
exam_scores = np.random.normal(70, 10, n_students)

# Cutoff for scholarship
cutoff = 70
scholarship = (exam_scores >= cutoff).astype(int)

# True scholarship effect on GPA
scholarship_effect = 0.4  # Scholarship increases GPA by 0.4 points

# GPA depends on exam score (continuous relationship) + scholarship effect
# Baseline: GPA increases with exam performance
gpa = (
    2.0
    + 0.02 * exam_scores
    + scholarship_effect * scholarship
    + np.random.normal(0, 0.3, n_students)
)

# Ensure GPA is in valid range [0, 4]
gpa = np.clip(gpa, 0, 4)

# Create dataframe
df_rdd = pd.DataFrame(
    {
        "Exam_Score": exam_scores,
        "Scholarship": scholarship,
        "GPA": gpa,
        "Distance_from_Cutoff": exam_scores - cutoff,
    }
)

print("Regression Discontinuity Data:")
print(df_rdd.head(10))

# Analyze effect near the cutoff
bandwidth = 5  # Look at students within ±5 points of cutoff
df_near_cutoff = df_rdd[np.abs(df_rdd["Distance_from_Cutoff"]) <= bandwidth]

gpa_just_above = df_near_cutoff[df_near_cutoff["Scholarship"] == 1]["GPA"]
gpa_just_below = df_near_cutoff[df_near_cutoff["Scholarship"] == 0]["GPA"]

t_stat, p_value = ttest_ind(gpa_just_above, gpa_just_below)

print(f"\n📊 RDD Analysis (within {bandwidth} points of cutoff):")
print(f"\nStudents just above cutoff (scholarship): Mean GPA = {gpa_just_above.mean():.3f}")
print(f"Students just below cutoff (no scholarship): Mean GPA = {gpa_just_below.mean():.3f}")
print(
    f"Estimated scholarship effect: {gpa_just_above.mean() - gpa_just_below.mean():.3f} GPA points"
)
print(f"\nt-test: t = {t_stat:.3f}, p = {p_value:.4f}")

if p_value < 0.05:
    print(f"\n✓ Scholarship has a significant positive effect on GPA")
else:
    print(f"\n✗ No significant scholarship effect detected")

In [ ]:
# Visualize regression discontinuity
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Panel 1: Full scatter plot
colors = ["#E63946" if s == 0 else "#06A77D" for s in df_rdd["Scholarship"]]
axes[0].scatter(df_rdd["Exam_Score"], df_rdd["GPA"], c=colors, alpha=0.4, s=50, edgecolors="none")

# Fit separate regression lines on each side
below_cutoff = df_rdd[df_rdd["Exam_Score"] < cutoff]
above_cutoff = df_rdd[df_rdd["Exam_Score"] >= cutoff]

if len(below_cutoff) > 0:
    slope_below, intercept_below, _, _, _ = linregress(
        below_cutoff["Exam_Score"], below_cutoff["GPA"]
    )
    x_below = np.linspace(below_cutoff["Exam_Score"].min(), cutoff, 100)
    y_below = slope_below * x_below + intercept_below
    axes[0].plot(x_below, y_below, color="#E63946", linewidth=3, label="No Scholarship")

if len(above_cutoff) > 0:
    slope_above, intercept_above, _, _, _ = linregress(
        above_cutoff["Exam_Score"], above_cutoff["GPA"]
    )
    x_above = np.linspace(cutoff, above_cutoff["Exam_Score"].max(), 100)
    y_above = slope_above * x_above + intercept_above
    axes[0].plot(x_above, y_above, color="#06A77D", linewidth=3, label="Scholarship")

# Mark discontinuity
axes[0].axvline(x=cutoff, color="black", linestyle="--", linewidth=2, label="Cutoff (70)")

axes[0].set_xlabel("Entrance Exam Score", fontsize=12, fontweight="bold")
axes[0].set_ylabel("College GPA", fontsize=12, fontweight="bold")
axes[0].set_title("Regression Discontinuity Design", fontsize=13, fontweight="bold")
axes[0].legend(loc="lower right")
axes[0].grid(True, alpha=0.3)

# Panel 2: Zoomed in near cutoff
zoom_range = 10
df_zoom = df_rdd[
    (df_rdd["Exam_Score"] >= cutoff - zoom_range) & (df_rdd["Exam_Score"] <= cutoff + zoom_range)
]

colors_zoom = ["#E63946" if s == 0 else "#06A77D" for s in df_zoom["Scholarship"]]
axes[1].scatter(
    df_zoom["Exam_Score"],
    df_zoom["GPA"],
    c=colors_zoom,
    alpha=0.6,
    s=80,
    edgecolors="black",
    linewidths=0.5,
)

# Fit lines in zoom window
below_zoom = df_zoom[df_zoom["Exam_Score"] < cutoff]
above_zoom = df_zoom[df_zoom["Exam_Score"] >= cutoff]

if len(below_zoom) > 0:
    slope_bz, intercept_bz, _, _, _ = linregress(below_zoom["Exam_Score"], below_zoom["GPA"])
    x_bz = np.linspace(cutoff - zoom_range, cutoff, 50)
    y_bz = slope_bz * x_bz + intercept_bz
    axes[1].plot(x_bz, y_bz, color="#E63946", linewidth=3)

if len(above_zoom) > 0:
    slope_az, intercept_az, _, _, _ = linregress(above_zoom["Exam_Score"], above_zoom["GPA"])
    x_az = np.linspace(cutoff, cutoff + zoom_range, 50)
    y_az = slope_az * x_az + intercept_az
    axes[1].plot(x_az, y_az, color="#06A77D", linewidth=3)

# Mark discontinuity and effect size
axes[1].axvline(x=cutoff, color="black", linestyle="--", linewidth=2)

# Calculate discontinuity size at cutoff
if len(below_zoom) > 0 and len(above_zoom) > 0:
    y_below_at_cutoff = slope_bz * cutoff + intercept_bz
    y_above_at_cutoff = slope_az * cutoff + intercept_az
    discontinuity = y_above_at_cutoff - y_below_at_cutoff

    # Draw arrow showing discontinuity
    axes[1].annotate(
        "",
        xy=(cutoff + 0.5, y_above_at_cutoff),
        xytext=(cutoff + 0.5, y_below_at_cutoff),
        arrowprops=dict(arrowstyle="<->", color="red", lw=2),
    )
    axes[1].text(
        cutoff + 1,
        (y_above_at_cutoff + y_below_at_cutoff) / 2,
        f"Effect:\n{discontinuity:.3f}",
        fontsize=10,
        fontweight="bold",
    )

axes[1].set_xlabel("Entrance Exam Score", fontsize=12, fontweight="bold")
axes[1].set_ylabel("College GPA", fontsize=12, fontweight="bold")
axes[1].set_title(f"Zoomed: ±{zoom_range} Points from Cutoff", fontsize=13, fontweight="bold")
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs/module_03/rdd_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n💡 The discontinuity at the cutoff reveals the causal effect of the scholarship.")
print("   Students just above and below the cutoff are nearly identical, except for treatment.")

### RDD Assumptions and Validity

**Key Assumption**: Units cannot precisely manipulate their running variable to get treatment.

**Validity Checks**:
1. **McCrary density test**: Check for suspicious bunching at cutoff
2. **Covariate balance**: Pre-treatment variables should be continuous at cutoff
3. **Placebo cutoffs**: No discontinuity at arbitrary thresholds
4. **Bandwidth sensitivity**: Results should be stable across bandwidths

**Strengths**:
✓ Provides credible causal estimates  
✓ Transparent and intuitive  
✓ No need for randomization  

**Limitations**:
✗ Only applies at the cutoff (limited external validity)  
✗ Requires large samples near cutoff  
✗ Vulnerable to manipulation  

## 6. Natural Experiments

**Natural experiments** occur when external events create quasi-random treatment assignment.

### Classic Examples

1. **Vietnam Draft Lottery** (Angrist, 1990)
   - Random assignment: Birthdate lottery → Draft status
   - Outcome: Lifetime earnings
   - Finding: Military service reduced earnings

2. **London Cholera Outbreak** (Snow, 1854)
   - Natural variation: Two water companies served overlapping areas
   - One company drew water upstream (clean), other downstream (contaminated)
   - Outcome: Cholera deaths
   - Finding: Contaminated water caused cholera

3. **Minimum Wage Increases** (Card & Krueger, 1994)
   - Policy change: New Jersey raised minimum wage, Pennsylvania did not
   - Outcome: Fast food employment
   - Finding: No decrease in employment

### When to Use Natural Experiments
- When randomization is impossible or unethical
- When policy changes create comparison groups
- When geographic or temporal variation exists

### Example: School Closing Policy (Simulated)

Two neighboring districts: One closes schools during flu season, the other doesn't.  
**Question**: Does school closure reduce flu transmission?

In [ ]:
# Simulate natural experiment: school closure effect on flu rates

np.random.seed(101)
n_weeks = 20

# Week numbers
weeks = np.arange(1, n_weeks + 1)

# Closure happens in week 11
closure_week = 11

# District A: Implements school closure
# Flu rates rise naturally, then drop after closure
district_A_before = 50 + 5 * np.arange(1, closure_week) + np.random.normal(0, 5, closure_week - 1)
district_A_after = (
    70
    - 3 * np.arange(1, n_weeks - closure_week + 2)
    + np.random.normal(0, 5, n_weeks - closure_week + 1)
)
district_A_flu = np.concatenate([district_A_before, district_A_after])

# District B: No school closure (control)
# Flu rates continue rising
district_B_flu = 50 + 5 * np.arange(1, n_weeks + 1) + np.random.normal(0, 5, n_weeks)

# Create dataframe
df_natural = pd.DataFrame(
    {
        "Week": np.tile(weeks, 2),
        "District": np.repeat(["A (Closure)", "B (Control)"], n_weeks),
        "Flu_Rate": np.concatenate([district_A_flu, district_B_flu]),
        "Intervention": np.tile(weeks >= closure_week, 2),
    }
)

print("Natural Experiment Data:")
print(df_natural.head(15))

In [ ]:
# Visualize natural experiment
fig, ax = plt.subplots(figsize=(12, 7))

for district, color, marker in [("A (Closure)", "#06A77D", "o"), ("B (Control)", "#E63946", "s")]:
    subset = df_natural[df_natural["District"] == district]
    ax.plot(
        subset["Week"],
        subset["Flu_Rate"],
        marker=marker,
        linestyle="-",
        linewidth=2,
        markersize=8,
        color=color,
        label=district,
        alpha=0.8,
    )

# Mark intervention
ax.axvline(
    x=closure_week,
    color="black",
    linestyle="--",
    linewidth=2,
    label="School Closure (District A only)",
)
ax.axvspan(0, closure_week, alpha=0.05, color="gray")
ax.axvspan(closure_week, n_weeks, alpha=0.1, color="green")

ax.set_xlabel("Week", fontsize=13, fontweight="bold")
ax.set_ylabel("Flu Rate (cases per 10,000)", fontsize=13, fontweight="bold")
ax.set_title(
    "Natural Experiment: School Closure Effect on Flu Transmission", fontsize=14, fontweight="bold"
)
ax.legend(loc="upper left", fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs/module_03/natural_experiment.png", dpi=300, bbox_inches="tight")
plt.show()

# Difference-in-differences analysis
A_before = df_natural[
    (df_natural["District"] == "A (Closure)") & (df_natural["Week"] < closure_week)
]["Flu_Rate"].mean()
A_after = df_natural[
    (df_natural["District"] == "A (Closure)") & (df_natural["Week"] >= closure_week)
]["Flu_Rate"].mean()
B_before = df_natural[
    (df_natural["District"] == "B (Control)") & (df_natural["Week"] < closure_week)
]["Flu_Rate"].mean()
B_after = df_natural[
    (df_natural["District"] == "B (Control)") & (df_natural["Week"] >= closure_week)
]["Flu_Rate"].mean()

# Difference-in-differences estimate
did_estimate = (A_after - A_before) - (B_after - B_before)

print(f"\n📊 Difference-in-Differences Analysis:")
print(f"\nDistrict A (Closure):")
print(f"  Before: {A_before:.2f} cases")
print(f"  After:  {A_after:.2f} cases")
print(f"  Change: {A_after - A_before:.2f} cases")

print(f"\nDistrict B (Control):")
print(f"  Before: {B_before:.2f} cases")
print(f"  After:  {B_after:.2f} cases")
print(f"  Change: {B_after - B_before:.2f} cases")

print(f"\n📐 Difference-in-Differences Estimate: {did_estimate:.2f} cases")
print(f"\n💡 School closure reduced flu transmission by {abs(did_estimate):.2f} cases per 10,000,")
print(f"   after accounting for the natural trend (control district).")

## 7. Sensitivity Analysis

**Sensitivity analysis** tests how robust your findings are to:
1. Different analytical choices
2. Violations of assumptions
3. Unmeasured confounding

### Types of Sensitivity Analysis

#### 1. Model Specification
- Try different functional forms (linear, quadratic, log)
- Include/exclude covariates
- Use different estimation methods

#### 2. Sample Restrictions
- Drop outliers
- Restrict to subgroups
- Vary bandwidth (in RDD)

#### 3. Unmeasured Confounding
- Calculate how strong a confounder would need to be to eliminate effect
- E-value: Minimum strength of confounding to explain away result

### Example: Testing Robustness of Treatment Effect

In [ ]:
# Sensitivity analysis: How robust is our finding?

# Simulate treatment effect study
np.random.seed(202)
n = 200

# Treatment assignment (e.g., training program)
treatment = np.random.binomial(1, 0.5, n)

# Outcome (e.g., test score)
true_effect = 10  # Training increases scores by 10 points
outcome = 50 + true_effect * treatment + np.random.normal(0, 15, n)

# Observed effect
treated_scores = outcome[treatment == 1]
control_scores = outcome[treatment == 0]
observed_effect = treated_scores.mean() - control_scores.mean()

print(f"Observed treatment effect: {observed_effect:.2f} points")

# Sensitivity Analysis 1: Effect of unmeasured confounder
print("\n" + "=" * 70)
print("SENSITIVITY TO UNMEASURED CONFOUNDING")
print("=" * 70)

print("\nHow strong would a confounder need to be to eliminate the effect?\n")

# Simulate different confounding scenarios
confounder_strengths = np.arange(0, 0.6, 0.1)
adjusted_effects = []

for strength in confounder_strengths:
    # Confounder affects both treatment and outcome
    confounder = np.random.normal(0, 1, n)

    # Treatment probability depends on confounder
    treatment_conf = np.random.binomial(1, 0.5 + strength * (confounder / confounder.std()), n)

    # Outcome depends on treatment and confounder
    outcome_conf = (
        50 + true_effect * treatment + 20 * strength * confounder + np.random.normal(0, 15, n)
    )

    # Naive estimate (ignoring confounder)
    treated_conf = outcome_conf[treatment_conf == 1]
    control_conf = outcome_conf[treatment_conf == 0]

    if len(treated_conf) > 0 and len(control_conf) > 0:
        biased_effect = treated_conf.mean() - control_conf.mean()
        adjusted_effects.append(biased_effect)
    else:
        adjusted_effects.append(np.nan)

# Create sensitivity table
sensitivity_df = pd.DataFrame(
    {
        "Confounder_Strength": confounder_strengths,
        "Biased_Effect_Estimate": adjusted_effects,
        "Percent_Bias": [
            (e - observed_effect) / observed_effect * 100 if not np.isnan(e) else np.nan
            for e in adjusted_effects
        ],
    }
)

print(sensitivity_df.to_string(index=False))
print("\n💡 This shows how effect estimates change under different confounding scenarios.")

In [ ]:
# Visualize sensitivity analysis
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(
    confounder_strengths,
    adjusted_effects,
    "o-",
    linewidth=3,
    markersize=10,
    color="#E63946",
    label="Biased Estimate",
)
ax.axhline(
    y=observed_effect,
    color="#06A77D",
    linestyle="--",
    linewidth=2,
    label=f"Original Estimate ({observed_effect:.2f})",
)
ax.axhline(y=0, color="gray", linestyle=":", linewidth=1.5, label="No Effect")

ax.fill_between(confounder_strengths, 0, adjusted_effects, alpha=0.2, color="#E63946")

ax.set_xlabel("Strength of Unmeasured Confounder", fontsize=13, fontweight="bold")
ax.set_ylabel("Estimated Treatment Effect", fontsize=13, fontweight="bold")
ax.set_title(
    "Sensitivity Analysis: Effect of Unmeasured Confounding", fontsize=14, fontweight="bold"
)
ax.legend(loc="upper right", fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig("outputs/module_03/sensitivity_analysis.png", dpi=300, bbox_inches="tight")
plt.show()

print("\n📊 Interpretation:")
print("   As unmeasured confounding increases, the treatment effect estimate shrinks.")
print("   A moderately strong confounder (≥0.3) could eliminate the observed effect.")
print("\n💡 This analysis helps researchers understand the robustness of their findings.")

## 8. Practice Exercises

Apply what you've learned to solidify your understanding.

### Exercise 1: Design Selection

For each scenario, select the most appropriate design:

1. **Scenario**: You want to test if a new teaching method improves student performance, but you can only recruit 30 students.
   - **Answer**: ____________

2. **Scenario**: A state implements a seatbelt law in 2020. You have monthly traffic fatality data from 2015-2023.
   - **Answer**: ____________

3. **Scenario**: College admission is based on entrance exam scores. Students scoring ≥80 are admitted.
   - **Answer**: ____________

4. **Scenario**: Two medications for hypertension, each taken for 4 weeks. You want to compare them.
   - **Answer**: ____________

In [ ]:
# Exercise 2: Power Calculation
# Calculate required sample size for a within-subjects design
# Target: 85% power, alpha = 0.05, expected effect size d = 0.4

# YOUR CODE HERE
target_power = 0.85
alpha = 0.05
effect_size = 0.4

# Hint: Use the simulation function from earlier
# Test different sample sizes until you reach target power

# Example starter:
# for n in range(20, 100, 5):
#     power = simulate_experiment(n, effect_size, 'within', 500)
#     if power >= target_power:
#         print(f"Required sample size: {n}")
#         break

In [ ]:
# Exercise 3: ITS Analysis
# You have monthly crime data before and after a policing intervention
# Analyze whether the intervention reduced crime

# Generate data
np.random.seed(999)
months = np.arange(1, 37)  # 36 months
intervention_month = 19

# Crime rate decreases after intervention (month 19)
crime_before = (
    100 + 2 * np.arange(1, intervention_month) + np.random.normal(0, 5, intervention_month - 1)
)
crime_after = (
    130
    - 3 * np.arange(1, len(months) - intervention_month + 2)
    + np.random.normal(0, 5, len(months) - intervention_month + 1)
)
crime_rate = np.concatenate([crime_before, crime_after])

# YOUR TASK:
# 1. Create appropriate variables (time, intervention dummy, time_after)
# 2. Fit regression model
# 3. Interpret coefficients
# 4. Visualize results

# YOUR CODE HERE

## 9. Summary and Key Takeaways

### Design Comparison Matrix

| Design | Causal Strength | Required Sample | Key Assumption | Best Use Case |
|--------|----------------|-----------------|----------------|---------------|
| **RCT (Between)** | ⭐⭐⭐⭐⭐ | Large | Random assignment | Gold standard when feasible |
| **RCT (Within)** | ⭐⭐⭐⭐⭐ | Small-Medium | No carryover | Limited samples, reversible treatments |
| **Crossover** | ⭐⭐⭐⭐ | Small | Adequate washout | Chronic conditions, medications |
| **ITS** | ⭐⭐⭐ | Time series data | No concurrent events | Policy evaluation |
| **RDD** | ⭐⭐⭐⭐ | Large (near cutoff) | No manipulation | Threshold-based assignment |
| **Natural Experiment** | ⭐⭐⭐ | Varies | Exogenous variation | When randomization impossible |

### Decision Framework

```
Can you randomize? ──YES──> RCT
        │                     │
       NO                     ├─ Large sample? ──> Between-subjects
        │                     └─ Small sample? ──> Within-subjects/Crossover
        │
        ├─ Is there a cutoff/threshold? ──YES──> Regression Discontinuity
        │
        ├─ Is there time series data? ──YES──> Interrupted Time Series
        │
        ├─ Is there natural variation? ──YES──> Natural Experiment
        │
        └─ Otherwise ──> Observational + Causal Inference Methods
```

### Critical Reminders

1. **No design is perfect**: Every design has trade-offs between internal validity, external validity, and feasibility.

2. **Transparency is key**: Clearly document your design choices, assumptions, and limitations.

3. **Always check assumptions**: Violation of key assumptions can invalidate causal claims.

4. **Conduct sensitivity analyses**: Test how robust your findings are to different specifications.

5. **Match design to question**: Let your research question guide design selection, not convenience.

6. **Power matters**: Underpowered studies waste resources and produce unreliable results.

### Moving Forward

You now have a sophisticated toolkit for causal inference when perfect experiments aren't possible. The next module will cover **survey design and measurement**, essential skills for collecting high-quality data in any research design.

## 10. Additional Resources

### Essential Readings

1. **Shadish, Cook, & Campbell (2002)**. *Experimental and Quasi-Experimental Designs for Generalized Causal Inference*
   - The definitive guide to experimental design

2. **Angrist & Pischke (2009)**. *Mostly Harmless Econometrics*
   - Practical guide to causal inference in observational studies

3. **Bernal, Cummins, & Gasparrini (2017)**. "Interrupted time series regression for the evaluation of public health interventions"
   - Modern ITS methods

4. **Lee & Lemieux (2010)**. "Regression Discontinuity Designs in Economics"
   - Comprehensive RDD tutorial

### Online Tools

- **Power calculators**: G*Power (free software)
- **RDD visualization**: rdrobust package (R/Stata)
- **ITS analysis**: itsa package (Stata), nlme (R)

### Practice Datasets

- **NHANES**: National Health and Nutrition Examination Survey
- **IPUMS**: Census and survey data
- **Dataverse**: Research data repository

In [ ]:
# Save a design selection checklist
checklist = pd.DataFrame(
    {
        "Question": [
            "1. Can you randomize participants to conditions?",
            "2. Are there carryover or practice effects?",
            "3. Is the treatment reversible?",
            "4. Is there a clear assignment cutoff/threshold?",
            "5. Do you have time series data pre/post intervention?",
            "6. Are there natural sources of variation?",
            "7. What is your sample size?",
            "8. What are ethical constraints?",
            "9. What is your budget?",
            "10. What is the expected effect size?",
        ],
        "Implication": [
            "YES → RCT possible | NO → Quasi-experimental",
            "YES → Between-subjects | NO → Within-subjects ok",
            "YES → Crossover possible | NO → Between-subjects",
            "YES → Consider RDD | NO → Other designs",
            "YES → Consider ITS | NO → Other designs",
            "YES → Consider natural experiment | NO → RCT needed",
            "Small → Within-subjects | Large → Between-subjects",
            "Restricts randomization options",
            "Low → Favor within-subjects or quasi-experimental",
            "Small → Need larger N for adequate power",
        ],
    }
)

checklist.to_csv("outputs/module_03/design_selection_checklist.csv", index=False)
print("✓ Design selection checklist saved to outputs/module_03/")
print("\n" + checklist.to_string(index=False))

---

## Congratulations!

You've completed **Module 03: Advanced Experimental Designs**. You can now:

✓ Select appropriate experimental designs for different research contexts  
✓ Understand power advantages of within-subjects designs  
✓ Implement and analyze crossover trials  
✓ Conduct interrupted time series analyses  
✓ Apply regression discontinuity designs  
✓ Identify and leverage natural experiments  
✓ Perform sensitivity analyses to test robustness  

**Next Module**: Survey Design & Measurement  
**File**: `04_survey_design_measurement.ipynb`

---